In [11]:
import os
import utils.read_utils as hlpread
from config.config import CONFIGURATION_PATH
from itertools import permutations, combinations

In [3]:
current_dir = os.path.dirname(os.path.realpath('__file__'))
PROJECT_ROOT = os.path.abspath(os.path.join(current_dir, os.pardir))
PROJECT_ROOT

train_data = os.path.join(PROJECT_ROOT, 
                          hlpread.read_yaml_key(os.path.join(PROJECT_ROOT, CONFIGURATION_PATH),
                                                'data_source',
                                                'data_folders'),
                          hlpread.read_yaml_key(os.path.join(PROJECT_ROOT, CONFIGURATION_PATH),
                                                'data_source',
                                                'clean_train'),
                        )

train_data

'f:\\github_workspace\\employee_access_resource_classifier\\data\\clean_train.csv'

In [19]:
db_train = hlpread.read_from_parquet(train_data)

print(db_train.shape)
db_train.head()

(30391, 10)


,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [20]:
target_col = 'ACTION'
col_use = [x for x in db_train.columns if not x in [target_col, 'ROLE_TITLE', 'MGR_ID']] 
#['ACTION', 'RESOURCE',  "ROLE_ROLLUP_2", 'ROLE_DEPTNAME','ROLE_FAMILY_DESC', 'ROLE_CODE']
col_use

['RESOURCE',
 'ROLE_ROLLUP_1',
 'ROLE_ROLLUP_2',
 'ROLE_DEPTNAME',
 'ROLE_FAMILY_DESC',
 'ROLE_FAMILY',
 'ROLE_CODE']

In [14]:
db_train[col_use].nunique()

RESOURCE            7518
ROLE_ROLLUP_1        128
ROLE_ROLLUP_2        177
ROLE_DEPTNAME        449
ROLE_FAMILY_DESC    2358
ROLE_FAMILY           67
ROLE_CODE            343
dtype: int64

In [21]:
db_train[col_use] = db_train[col_use].applymap(str)
db_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30391 entries, 0 to 32768
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ACTION            30391 non-null  int64 
 1   RESOURCE          30391 non-null  object
 2   MGR_ID            30391 non-null  int64 
 3   ROLE_ROLLUP_1     30391 non-null  object
 4   ROLE_ROLLUP_2     30391 non-null  object
 5   ROLE_DEPTNAME     30391 non-null  object
 6   ROLE_TITLE        30391 non-null  int64 
 7   ROLE_FAMILY_DESC  30391 non-null  object
 8   ROLE_FAMILY       30391 non-null  object
 9   ROLE_CODE         30391 non-null  object
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [22]:
i=1
for c1,c2 in permutations(col_use, 2):
    print(f'{i}: {c1}-{c2}')
    i = i+1
    print('='*50)

1: RESOURCE-ROLE_ROLLUP_1
2: RESOURCE-ROLE_ROLLUP_2
3: RESOURCE-ROLE_DEPTNAME
4: RESOURCE-ROLE_FAMILY_DESC
5: RESOURCE-ROLE_FAMILY
6: RESOURCE-ROLE_CODE
7: ROLE_ROLLUP_1-RESOURCE
8: ROLE_ROLLUP_1-ROLE_ROLLUP_2
9: ROLE_ROLLUP_1-ROLE_DEPTNAME
10: ROLE_ROLLUP_1-ROLE_FAMILY_DESC
11: ROLE_ROLLUP_1-ROLE_FAMILY
12: ROLE_ROLLUP_1-ROLE_CODE
13: ROLE_ROLLUP_2-RESOURCE
14: ROLE_ROLLUP_2-ROLE_ROLLUP_1
15: ROLE_ROLLUP_2-ROLE_DEPTNAME
16: ROLE_ROLLUP_2-ROLE_FAMILY_DESC
17: ROLE_ROLLUP_2-ROLE_FAMILY
18: ROLE_ROLLUP_2-ROLE_CODE
19: ROLE_DEPTNAME-RESOURCE
20: ROLE_DEPTNAME-ROLE_ROLLUP_1
21: ROLE_DEPTNAME-ROLE_ROLLUP_2
22: ROLE_DEPTNAME-ROLE_FAMILY_DESC
23: ROLE_DEPTNAME-ROLE_FAMILY
24: ROLE_DEPTNAME-ROLE_CODE
25: ROLE_FAMILY_DESC-RESOURCE
26: ROLE_FAMILY_DESC-ROLE_ROLLUP_1
27: ROLE_FAMILY_DESC-ROLE_ROLLUP_2
28: ROLE_FAMILY_DESC-ROLE_DEPTNAME
29: ROLE_FAMILY_DESC-ROLE_FAMILY
30: ROLE_FAMILY_DESC-ROLE_CODE
31: ROLE_FAMILY-RESOURCE
32: ROLE_FAMILY-ROLE_ROLLUP_1
33: ROLE_FAMILY-ROLE_ROLLUP_2
34: ROLE_FAMIL

In [7]:
new_col4train = []

col4train = col_use
col4train.remove('RESOURCE')

for c1,c2 in permutations(col4train, 2):
    
    name = "{}_{}".format(c1,c2)
    new_col4train.append(name)
    db_train[name] = db_train[c1] + " " + db_train[c2]

db_train.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,...,ROLE_ROLLUP_2_ROLE_DEPTNAME,ROLE_ROLLUP_2_ROLE_FAMILY_DESC,ROLE_ROLLUP_2_ROLE_FAMILY,ROLE_ROLLUP_2_ROLE_CODE,ROLE_DEPTNAME_ROLE_FAMILY_DESC,ROLE_DEPTNAME_ROLE_FAMILY,ROLE_DEPTNAME_ROLE_CODE,ROLE_FAMILY_DESC_ROLE_FAMILY,ROLE_FAMILY_DESC_ROLE_CODE,ROLE_FAMILY_ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908,...,118300 123472,118300 117906,118300 290919,118300 117908,123472 117906,123472 290919,123472 117908,117906 290919,117906 117908,290919 117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539,...,118343 123125,118343 118536,118343 308574,118343 118539,123125 118536,123125 308574,123125 118539,118536 308574,118536 118539,308574 118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880,...,118220 117884,118220 267952,118220 19721,118220 117880,117884 267952,117884 19721,117884 117880,267952 19721,267952 117880,19721 117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322,...,118343 119993,118343 240983,118343 290919,118343 118322,119993 240983,119993 290919,119993 118322,240983 290919,240983 118322,290919 118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325,...,117930 119569,117930 123932,117930 19793,117930 119325,119569 123932,119569 19793,119569 119325,123932 19793,123932 119325,19793 119325


In [8]:
db_train.shape

(30391, 25)

In [9]:
nq = db_train[new_col4train].nunique()
nq[nq.argsort()]

ROLE_ROLLUP_1_ROLE_ROLLUP_2        187
ROLE_FAMILY_ROLE_CODE              343
ROLE_ROLLUP_1_ROLE_FAMILY          750
ROLE_ROLLUP_2_ROLE_FAMILY         1045
ROLE_ROLLUP_1_ROLE_DEPTNAME       1185
ROLE_ROLLUP_2_ROLE_DEPTNAME       1398
ROLE_ROLLUP_1_ROLE_CODE           1506
ROLE_DEPTNAME_ROLE_FAMILY         1613
ROLE_ROLLUP_2_ROLE_CODE           2125
ROLE_FAMILY_DESC_ROLE_FAMILY      2586
ROLE_DEPTNAME_ROLE_CODE           2946
ROLE_FAMILY_DESC_ROLE_CODE        3013
ROLE_ROLLUP_1_ROLE_FAMILY_DESC    3282
ROLE_ROLLUP_2_ROLE_FAMILY_DESC    3949
ROLE_DEPTNAME_ROLE_FAMILY_DESC    4553
dtype: int64

Observation:
* Combination of some feature have high cardinality. Combining such catagories together reduces the number of observation per categoy in the resulting features. It will not generalize well and the model might overfit.
* KFoldTargetEncoding will not work in case of "RESOURCE"  as 89% of records is single entry. In this case most of the time in each fold the value will be NaN and due to this this will be always be replace to mean of the "ACTION" which will not be helpful. KFold is helfull if in each folder will have the group value in the validation dataset.
Alternative is compute the mean target X_tr of the KFold split and replace the missing map in X_val of the KFold split with the mean.  

In [10]:
db_train.groupby(['ROLE_ROLLUP_1_ROLE_ROLLUP_2'])['ROLE_FAMILY_ROLE_CODE'].agg(lambda x: " ".join(list([str(y) for y in x])))

ROLE_ROLLUP_1_ROLE_ROLLUP_2
11146 118491     19721 118570 117887 117888 117887 117973 19721...
117876 117877    19721 117880 19721 118570 19721 247660 19793 1...
117882 117883    19721 117880 117887 117888 19721 117880 117887...
117882 123999    117887 118055 117887 118055 117887 118055 1178...
117887 118178    118638 119851 292795 118943 292795 118943 1219...
                                       ...                        
311178 119256                          118474 136061 118474 136061
4292 124335      119772 130062 119184 128095 119184 128095 1191...
4292 126095      117887 117973 117887 117973 119184 119194 1191...
5110 117954      19721 118570 19721 118570 118870 124539 19721 ...
91261 118026     118205 119964 290919 118322 290919 118279 2909...
Name: ROLE_FAMILY_ROLE_CODE, Length: 187, dtype: object

In [ ]:
db_train.groupby(['ROLE_ROLLUP_1'])['ROLE_DEPTNAME'].agg(lambda x: " ".join(list([str(y) for y in x])))

: 